In [2]:
# Import libraries
import docx
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import scispacy
nlp = spacy.load("en_core_sci_lg")
from spacy import displacy
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import Doc, Span, Token
from summa import summarizer
import matplotlib.pyplot as plt
import os
from sklearn.decomposition import PCA

In [3]:
import benepar
#benepar.download('benepar_en2')

/Users/otrejo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/otrejo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/otrejo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/otrejo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework

In [5]:
text = 'Amorphous oxide semiconductors (AOSs)—ternary or quaternary oxides of post‐transition metals such as In‐Sn‐O, Zn‐Sn‐O, or In‐Ga‐Zn‐O—have been known for a decade and have attracted a great deal of attention as they possess several technological advantages, including low‐temperature large‐area deposition, mechanical flexibility, smooth surfaces, and high carrier mobility that is an order of magnitude larger than that of amorphous silicon (a‐Si:H). Compared to their crystalline counterparts, the structure of AOSs is extremely sensitive to deposition conditions, stoichiometry, and composition, giving rise to a wide range of tunable optical and electrical properties. The large parameter space and the resulting complex deposition–structure–property relationships in AOSs make the currently available theoretical and experimental research data rather scattered and the design of new materials difficult. In this work, the key properties of several In‐based AOSs are studied as a function of cooling rates, oxygen stoichiometry, cation composition, or lattice strain. Based on a thorough comparison of the results of ab initio modeling, comprehensive structural analysis, accurate property calculations, and systematic experimental measurements, a four‐dimensional parameter space for AOSs is derived, serving as a solid foundation for property optimization in known AOSs and for design of next‐generation transparent amorphous semiconductors.'

In [3]:
doc = nlp("This is a sentence.")
displacy.render(doc, style="dep")

In [7]:
doc = nlp(text)
displacy.render(list(doc.sents)[3], style="dep")

In [4]:
from benepar.spacy_plugin import BeneparComponent
nlp.add_pipe(BeneparComponent("benepar_en2"))
doc = nlp(u"The time for action is now. It's never too late to do something.")
sent = list(doc.sents)[0]
print(sent._.parse_string)
#(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))

W0713 23:50:45.889328 140737073968064 deprecation_wrapper.py:119] From /Users/otrejo/anaconda3/lib/python3.7/site-packages/benepar/base_parser.py:197: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.

W0713 23:50:50.555316 140737073968064 deprecation_wrapper.py:119] From /Users/otrejo/anaconda3/lib/python3.7/site-packages/benepar/base_parser.py:202: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))


In [9]:
doc = nlp(text)
sent = list(doc.sents)[2]
print(sent._.parse_string)

(S (NP (NP (DT The) (JJ large) (NN parameter) (NN space)) (CC and) (NP (NP (DT the) (VBG resulting) (JJ complex) (NN deposition) (CC –) (NN structure) (CC –) (NN property) (NNS relationships)) (PP (IN in) (NP (NNP AOSs))))) (VP (VBP make) (S (S (NP (DT the) (ADJP (ADJP (RB currently) (JJ available)) (UCP (JJ theoretical) (CC and) (JJ experimental))) (NN research) (NNS data)) (ADJP (RB rather) (VBN scattered))) (CC and) (S (NP (NP (DT the) (NN design)) (PP (IN of) (NP (JJ new) (NNS materials)))) (ADJP (JJ difficult))))) (. .))


In [ ]:
#0 sentence
(S (NP (NP (NP (JJ Amorphous) (NN oxide) (NNS semiconductors)) (JJ (AOSs)—ternary) (CC or) (JJ quaternary) (NNS oxides)) (PP (IN of) (NP (NP (NN post‐transition) (NNS metals)) (PP (JJ such) (IN as) (NNP In‐Sn‐O) (, ,) (NNP Zn‐Sn‐O) (, ,) (CC or) (NP (NNP In‐Ga‐Zn‐O)) (, —))))) (VP (VP (VBP have) (VP (VBN been) (VP (VBN known) (PP (IN for) (NP (DT a) (NN decade)))))) (CC and) (VP (VBP have) (VP (VBN attracted) (NP (NP (DT a) (JJ great) (NN deal)) (PP (IN of) (NP (NN attention)))) (SBAR (IN as) (S (NP (PRP they)) (VP (VBP possess) (NP (NP (JJ several) (JJ technological) (NNS advantages)) (, ,) (PP (VBG including) (NP (NP (NN low‐temperature)) (NN large‐area) (NN deposition)) (, ,) (NP (JJ mechanical) (NN flexibility)) (, ,) (NP (JJ smooth) (NNS surfaces)) (, ,) (CC and) (NP (NP (JJ high) (NN carrier) (NN mobility)) (SBAR (WHNP (WDT that)) (S (VP (VBZ is) (ADJP (ADJP (NP (DT an) (NN order)) (PP (IN of) (NP (NN magnitude))) (ADJP (JJR larger))) (PP (IN than) (NP (NP (DT that)) (PP (IN of) (NP (NP (JJ amorphous) (NN silicon)) (PRN (-LRB- -LRB-) (NP (JJ a‐Si) (. :) (NN H)) (-RRB- -RRB-))))))))))))))))))) (. .))

In [ ]:
# 1 sentence
(S 
 (PP (VBN Compared) (PP (TO to) (NP (PRP$ their) (JJ crystalline) (NNS counterparts)))) (, ,) 
 (NP (NP (DT the) (NN structure)) (PP (IN of) (NP (NNP AOSs)))) 
 (VP (VBZ is) 
  (ADJP (RB extremely) (JJ sensitive) 
               (PP (TO to) (NP 
                            (NP (NN deposition) (NNS conditions)) (, ,) 
                            (NP (NN stoichiometry)) (, ,) (CC and) 
                            (NP (NN composition))))) (, ,) 
  (S (VP (VBG giving) (NP (NN rise)) 
      (PP (TO to) (NP (NP (DT a) (JJ wide) (NN range)) 
                   (PP (IN of) 
                    (NP (JJ tunable) (ADJP (JJ optical) (CC and) (JJ electrical)) (NNS properties)))))))) (. .))



In [10]:
displacy.render(list(doc.sents)[1], style="dep")

In [ ]:
# 2 sentence
(S (NP (NP (DT The) (JJ large) (NN parameter) (NN space)) (CC and) (NP (NP (DT the) (VBG resulting) (JJ complex) (NN deposition) (CC –) (NN structure) (CC –) (NN property) (NNS relationships)) (PP (IN in) (NP (NNP AOSs))))) (VP (VBP make) (S (S (NP (DT the) (ADJP (ADJP (RB currently) (JJ available)) (UCP (JJ theoretical) (CC and) (JJ experimental))) (NN research) (NNS data)) (ADJP (RB rather) (VBN scattered))) (CC and) (S (NP (NP (DT the) (NN design)) (PP (IN of) (NP (JJ new) (NNS materials)))) (ADJP (JJ difficult))))) (. .))